# 使用 VectorDB 结合 Langchain 进行 RAG

百度向量数据库 VectorDB 是一款纯自研高性能、高性价比、生态丰富且即开即用的向量数据库服务。支持多种索引类型和相似度算法，百亿级向量规模，毫秒级查询延迟。百度向量数据库不仅能配合大模型打造专业知识库，还可以应用于图片搜索，音乐推荐，文本分类等领域

在这篇教程中，我们会演示如何使用 Vector DB 搭配千帆 Python SDK，在 Langchain 中实现 RAG 功能

# 准备工作

首先，我们需要安装 Langchain, 千帆 Python SDK  以及 VectorDB 的相关 Pypi 依赖

In [1]:
! pip install -U langchain qianfan pymochow

  Using cached pdfplumber-0.11.0-py3-none-any.whl.metadata (39 kB)
  Using cached langchain_text_splitters-0.0.1-py3-none-any.whl.metadata (2.0 kB)
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
  Using cached pdfminer.six-20231228-py3-none-any.whl.metadata (4.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 506.7 kB/s eta 0:00:00a 0:00:01
  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 3.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 370.4/370.4 kB 6.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 5.9 MB/s eta 0:00:00
Using cached pdfplumber-0.11.0-py3-none-any.whl (56 kB)
Using cached pdfminer.six-20231228-py3-none-any.whl (5.6 MB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 5.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.

然后，我们还需要设置相关环境变量，以运行示例代码

In [9]:
import os
from pymochow.auth.bce_credentials import BceCredentials

# 定义配置信息
account = 'root'
api_key = 'api_key'
endpoint = 'ip_address'

# 初始化BceCredentials对象
credentials = BceCredentials(account, api_key)

# 设置千帆AI平台的安全认证信息（AK/SK），通过环境变量
# 注意替换以下参数为您的Access Key和Secret Key
os.environ['QIANFAN_ACCESS_KEY'] = 'your_console_access_key'
os.environ['QIANFAN_SECRET_KEY'] = 'your_console_secret_key'

# 创建数据库

在我们设置完基础信息之后，我们需要在 Vevtor DB 中创建相对应的向量数据库

In [10]:
import pymochow
from pymochow.configuration import Configuration

config_obj = Configuration(credentials=credentials, endpoint=endpoint)
client = pymochow.MochowClient(config_obj)

database_name = "document"

try:
    db = client.create_database(database_name)
except Exception as e:  # 捕获所有类型的异常
    print(f"Error: {e}")  # 打印异常信息

Error: HTTPConnectionPool(host='172.16.64.3', port=5287): Max retries exceeded with url: /v1/database?create= (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10aa8c370>: Failed to establish a new connection: [Errno 60] Operation timed out'))


以及创建相对应的向量数据库表

In [ ]:
import time

# 导入pymochow模型相关的类和枚举类型
from pymochow.model.schema import Schema, Field, VectorIndex, SecondaryIndex, HNSWParams
from pymochow.model.enum import FieldType, IndexType, MetricType, TableState
from pymochow.model.table import Partition

# 选择或创建数据库
db = client.database(database_name)

# 定义数据表的字段
fields = [
    Field("id", FieldType.UINT64, primary_key=True, partition_key=True, auto_increment=False, not_null=True),
    Field("text", FieldType.STRING),
    Field("metadata", FieldType.STRING),
    Field("source", FieldType.STRING),
    Field("vector", FieldType.FLOAT_VECTOR, not_null=True, dimension=384)
]

# 定义数据表的索引
indexes = [
    VectorIndex(index_name="vector_idx", field="vector", index_type=IndexType.HNSW, metric_type=MetricType.L2, params=HNSWParams(m=32, efconstruction=200)),
    SecondaryIndex(index_name="author_idx", field="author")
]

# 尝试创建数据表，捕获并打印可能出现的异常
table_name = "chunks"

try:
    table = db.create_table(table_name=table_name, replication=1, partition=Partition(partition_num=1), schema=Schema(fields=fields, indexes=indexes))
except Exception as e:  # 捕获所有类型的异常
    print(f"Error: {e}")  # 打印异常信息

# 轮询数据表状态，直到表状态为NORMAL，表示表已准备好
while True:
    time.sleep(2)  # 每次检查前暂停2秒，减少对服务器的压力
    table = db.describe_table(table_name)
    if table.state == TableState.NORMAL:  # 表状态为NORMAL，跳出循环
        break


# 准备向量数据

在经过上述的步骤之后，我们成功在 Vector DB 的实例中创建了一个数据表，可以在接下来的步骤中用于存储向量表示

在完成了向量数据库的创建之后，我们就可以开始尝试向向量数据库中添加数据了。为了演示，我们选择从网页上获取一篇知乎专栏，用于展示如何结合 Langchain 进行数据的向量化存储

In [ ]:
from langchain.document_loaders import WebBaseLoader  # 用于从网页中加载文档
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 用于文本分割
import qianfan  # 千帆AI平台SDK
import json
from pymochow.model.table import Row # 用于写入向量数据


# 加载PDF文档
loader = WebBaseLoader("https://zhuanlan.zhihu.com/p/85289282")  # 构建网页加载对象
documents = loader.load()  # 加载文档

# 设置文本分割器，指定分割的参数
# chunk_size定义了每个分割块的字符数，chunk_overlap定义了块之间的重叠字符数
# separators列表定义了用于分割的分隔符
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=384, 
    chunk_overlap=0, 
    separators=["\n\n", "\n", " ", "", "。", "，"]
)
all_splits = text_splitter.split_documents(documents)  # 对文档进行分割

# 初始化嵌入模型对象
# 为了避免请求过速碰到限流限制，我们设置 QPS = 5
emb = qianfan.Embedding(query_per_second=5)

embeddings = []  # 用于存储每个文本块的嵌入向量
for chunk in all_splits:  # 遍历所有分割的文本块
    # 获取文本块的嵌入向量，使用默认模型Embedding-V1
    resp = emb.do(texts=[chunk.page_content])
    embeddings.append(resp['data'][0]['embedding'])  # 将嵌入向量添加到列表中

# 逐行写入向量化数据
rows = []
for index, chunk in enumerate(all_splits):
    metadata = "{}"
    if chunk.metadata is not None:
        metadata = json.dumps(chunk.metadata)
    row = Row(
        id=index,
        text=chunk.page_content,
        metadata=metadata,
        source=chunk.metadata["source"],
        vector=embeddings[index]
    )
    rows.append(row)

# 选择或创建数据库
db = client.database(database_name)

try:
    table = db.describe_table(table_name)
    table.upsert(rows=rows) # 批量写入向量数据，一次最多支持写入1000条
    table.rebuild_index("vector_idx") # 创建向量索引，必要步骤
except Exception as e:  # 捕获所有类型的异常
    print(f"Error: {e}")  # 打印异常信息

# 使用向量数据库直接进行 RAG

当你完成上述两个步骤之后，你就有一个可以直接用于查询的云端向量数据库实例了。

此时，我们可以结合 Langchain 中集成的 Vevtor DB 以及千帆组件来实现在 Langchain 中配合 Vector DB 进行查询

In [ ]:
from langchain_community.vectorstores import BaiduVectorDB
from langchain_community.vectorstores.baiduvectordb import ConnectionParams, TableParams
from langchain_community.embeddings import QianfanEmbeddingsEndpoint
from langchain_community.chat_models import QianfanChatEndpoint
from langchain.chains import RetrievalQA

# 初始化向量嵌入和连接参数
embeddings = QianfanEmbeddingsEndpoint()
conn_params = ConnectionParams(
    endpoint=endpoint,
    account=account,
    api_key=api_key
)

# 初始化百度云向量数据库
vector_db = BaiduVectorDB(
    embedding=embeddings,
    connection_params=conn_params,
    table_params=TableParams(384),
    database_name=database_name,
    table_name=table_name,
    drop_old=False,
)

# 初始化检索器和对话模型
retriever = vector_db.as_retriever(search_type="similarity")
qianfan_chat_model = QianfanChatEndpoint(model="ERNIE-Bot", temperature=0.1)

# 初始化问答模块
qa = RetrievalQA.from_chain_type(llm=qianfan_chat_model, chain_type="refine", retriever=retriever, return_source_documents=True)


在接下来的部分中，我们可以尝试输入内容，来体验 RAG 的查询返回结果

In [ ]:
# 在 query 变量中输入你的问题
query = "明朝开国皇帝是谁"

res = qa(query)
answer, docs = res['result'], res['source_documents']

print(answer)
print(docs)